In [47]:
import os
os.chdir('C:/Users/islam/Downloads/kidney-Disease-classification-Deep-learning-project/research')

In [48]:
%pwd

'C:\\Users\\islam\\Downloads\\kidney-Disease-classification-Deep-learning-project\\research'

In [49]:
os.chdir("../")

In [50]:
%pwd

'C:\\Users\\islam\\Downloads\\kidney-Disease-classification-Deep-learning-project'

In [51]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [52]:
import sys
import os
import joblib

# Add the src folder to the Python path
project_dir = r'C:/Users/islam/Downloads/kidney-Disease-classification-Deep-learning-project/src'
sys.path.append(project_dir)

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [53]:
import os
from pathlib import Path

CONFIG_FILE_PATH = Path(os.path.abspath("config/config.yaml"))
PARAMS_FILE_PATH = Path(os.path.abspath("params.yaml"))

In [54]:
from pathlib import Path
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [19]:
pip install tensorflow==2.12.0

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.


  Using cached tensorflow_intel-2.12.0-cp38-cp38-win_amd64.whl.metadata (4.1 kB)
  Using cached numpy-1.23.5-cp38-cp38-win_amd64.whl.metadata (2.3 kB)
  Using cached wrapt-1.14.1-cp38-cp38-win_amd64.whl.metadata (6.7 kB)
  Using cached tensorboard-2.12.3-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorflow_estimator-2.12.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached keras-2.12.0-py2.py3-none-any.whl.metadata (1.4 kB)
Note: you may need to restart the kernel to use updated packages.
Using cached tensorflow_intel-2.12.0-cp38-cp38-win_amd64.whl (272.8 MB)
Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
Using cached numpy-1.23.5-cp38-cp38-win_amd64.whl (14.7 MB)
Using cached tensorboard-2.12.3-py3-none-any.whl (5.6 MB)
Using cached tensorflow_estimator-2.12.0-py2.py3-none-any.whl (440 kB)
Using cached wrapt-1.14.1-cp38-cp38-win_amd64.whl (35 kB)
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Success

In [55]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [56]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [57]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-10-28 15:50:47,419: INFO: common: yaml file: C:\Users\islam\Downloads\kidney-Disease-classification-Deep-learning-project\config\config.yaml loaded successfully]
[2024-10-28 15:50:47,465: INFO: common: yaml file: C:\Users\islam\Downloads\kidney-Disease-classification-Deep-learning-project\params.yaml loaded successfully]
[2024-10-28 15:50:47,502: INFO: common: created directory at: artifacts]
[2024-10-28 15:50:47,504: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-10-28 15:50:52,137: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)  